In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#import dask.dataframe as dd
import seaborn as sns
import random
import time
import tensorflow as tf
import math
from scipy import stats
import warnings

In [2]:
#Code Source : https://www.kaggle.com/arjanso/reducing-dataframe-memory-size-by-65
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df


def import_data(file):
    """create a dataframe and optimize its memory usage"""
    df = pd.read_csv(file, parse_dates=True, keep_date_col=True)
    df = reduce_mem_usage(df)
    return df

In [3]:
%%time
print('-' * 80)
print('train')
train_df = import_data('/content/drive/MyDrive/CaseStudy1/train.csv')

print('-' * 80)
print('test')
test_df = import_data('/content/drive/MyDrive/CaseStudy1/test.csv')


print('-' * 80)
print('store')
store_df = import_data('/content/drive/MyDrive/CaseStudy1/stores.csv')

print('-' * 80)
print('item')
item_df = import_data('/content/drive/MyDrive/CaseStudy1/items.csv')

--------------------------------------------------------------------------------
train


/usr/local/lib/python3.7/dist-packages/IPython/core/magic.py:188: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  call = lambda f, *a, **k: f(*a, **k)


Memory usage of dataframe is 5744.80 MB
Memory usage after optimization is: 1915.02 MB
Decreased by 66.7%
--------------------------------------------------------------------------------
test
Memory usage of dataframe is 106.07 MB
Memory usage after optimization is: 38.57 MB
Decreased by 63.6%
--------------------------------------------------------------------------------
store
Memory usage of dataframe is 0.00 MB
Memory usage after optimization is: 0.00 MB
Decreased by 4.6%
--------------------------------------------------------------------------------
item
Memory usage of dataframe is 0.13 MB
Memory usage after optimization is: 0.03 MB
Decreased by 73.8%
CPU times: user 59.4 s, sys: 10.5 s, total: 1min 9s
Wall time: 1min 21s


In [4]:
train_df.head(1)

,id,date,store_nbr,item_nbr,unit_sales,onpromotion
0,0,2013-01-01,25,103665,7.0,NaN


In [5]:
#Replacing True or False with 0,1 values in onpromotion and replacing the NAN values with 0(False)
train_df['onpromotion'] = train_df['onpromotion'].fillna(0)
train_df['onpromotion'] = train_df['onpromotion'].astype(int)

In [6]:
train_df['onpromotion'].value_counts()

0    117686418
1      7810622
Name: onpromotion, dtype: int64

In [7]:
#Replacing all the Unit Sales less than 0 to 0
train_df.unit_sales = np.where(train_df.unit_sales < 0, 0,train_df.unit_sales)

In [8]:
train_df['unit_sales'] = np.log1p(train_df['unit_sales'])

In [14]:
test_df.isnull().sum()

id             0
date           0
store_nbr      0
item_nbr       0
onpromotion    0
dtype: int64

In [13]:
train_df_final = pd.merge(train_df,item_df,on=['item_nbr'],how='left')
train_df_final = pd.merge(train_df_final,store_df,on=['store_nbr'],how='left')
train_df_final.head(1)

,id,date,store_nbr,item_nbr,unit_sales,onpromotion,family,class,perishable,city,state,type,cluster
0,0,2013-01-01,25,103665,2.079442,0,BREAD/BAKERY,2712,1,Salinas,Santa Elena,D,1


In [15]:
test_df_final = pd.merge(test_df,item_df,on=['item_nbr'],how='left')
test_df_final = pd.merge(test_df_final,store_df,on=['store_nbr'],how='left')
test_df_final.head(1)

,id,date,store_nbr,item_nbr,onpromotion,family,class,perishable,city,state,type,cluster
0,125497040,2017-08-16,1,96995,0.0,GROCERY I,1093,0,Quito,Pichincha,D,13


In [16]:
train_df_final['date'] = pd.to_datetime(train_df_final['date'])

In [17]:
train_df_final['year'] = np.uint16(train_df_final['date'].dt.year)
train_df_final['month'] = np.uint16(train_df_final['date'].dt.month)
train_df_final['day_of_week'] = np.uint8(train_df_final['date'].dt.dayofweek)
train_df_final.head(1)

,id,date,store_nbr,item_nbr,unit_sales,onpromotion,family,class,perishable,city,state,type,cluster,year,month,day_of_week
0,0,2013-01-01,25,103665,2.079442,0,BREAD/BAKERY,2712,1,Salinas,Santa Elena,D,1,2013,1,1


In [18]:
test_df_final['date'] = pd.to_datetime(test_df_final['date'])
test_df_final['year'] = np.uint16(test_df_final['date'].dt.year)
test_df_final['month'] = np.uint16(test_df_final['date'].dt.month)
test_df_final['day_of_week'] = np.uint8(test_df_final['date'].dt.dayofweek)
test_df_final.head(1)

,id,date,store_nbr,item_nbr,onpromotion,family,class,perishable,city,state,type,cluster,year,month,day_of_week
0,125497040,2017-08-16,1,96995,0.0,GROCERY I,1093,0,Quito,Pichincha,D,13,2017,8,2


In [21]:
test_df_final.drop(['date'], axis=1, inplace=True)
#train_df_final.drop(['date'], axis=1, inplace=True)

In [20]:
y = train_df_final['unit_sales'].values
x = train_df_final.drop(['unit_sales'], axis=1)

In [23]:
from sklearn.model_selection import train_test_split
x_train,x_cv,y_train,y_cv = train_test_split(x,y,test_size=0.2)

print(x_train.shape, y_train.shape)
print(x_cv.shape, y_cv.shape)
print(test_df_final.shape)

(100397632, 14) (100397632,)
(25099408, 14) (25099408,)
(3370464, 14)


In [24]:
from sklearn.externals import joblib

/usr/local/lib/python3.7/dist-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


In [26]:
joblib.dump(x_train, '/content/drive/MyDrive/CaseStudy1/x_train.pkl')
joblib.dump(x_cv, '/content/drive/MyDrive/CaseStudy1/x_cv.pkl')
joblib.dump(y_train, '/content/drive/MyDrive/CaseStudy1/y_train.pkl')
joblib.dump(y_cv, '/content/drive/MyDrive/CaseStudy1/y_cv.pkl')
joblib.dump(test_df_final, '/content/drive/MyDrive/CaseStudy1/test_df_final.pkl')
joblib.dump(test_df_final, '/content/drive/MyDrive/CaseStudy1/train_df_final.pkl')

['/content/drive/MyDrive/CaseStudy1/train_df_final.pkl']